<a href="https://colab.research.google.com/github/suneetsawant/Neural-Machine-Translation/blob/master/Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Machine Translation Using Attention 

This notebook covers implementation of converting English to  French using  Neural Network incorporated with attention mechanism .






## Import all neccessary libraries 

1. Tensorflow, Keras for implementing neural network architecture 
2. Pydrive to save-restore model on google drive 
3. NLTK to calculate BLEU Scores 

In [0]:
!pip install -U nltk   ## Updated nltk library for computing BLEU Scores
import numpy as np 
import pandas as pd 
import tensorflow as tf
 
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense,GRU,Embedding,GlobalMaxPooling2D,Dropout,add,CuDNNGRU,Activation

from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply,RepeatVector,Lambda,Reshape
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import load_model, Model
import keras as K
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import unicodedata
import sys
import gc
import re
import numpy as np
import os
import time
import nltk

%matplotlib inline
import matplotlib.pyplot as plt


tf.enable_eager_execution()
!pip install -U -q PyDrive     ##### For saving-downloading model to google drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
class Drive():   
  def __init__(self) :  
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    self.drive = GoogleDrive(gauth)


  def fileaction(self,files,op='up') : 
    file_list = self.drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
    for filename in files:
      flag = 0 

      for file1 in file_list:
        if (file1['title']) == filename :
            if (op == 'up'and flag==0) : 
                file1.Delete()
                self.upload(filename)
                flag = 1

            elif (op == 'down') : 
                self.download(filename,file1) 

      if(op=='up' and flag==0): 
            self.upload(filename)
            flag = 1

  def upload(self,filename) : 
      Uploadfile = self.drive.CreateFile({'title': filename})
      Uploadfile.SetContentFile(filename)
      Uploadfile.Upload()
      print("Saved '{}' to Drive".format(filename))

  def download(self,filename,file1): 
      downloaded = self.drive.CreateFile({'id':file1['id']})
      downloaded.GetContentFile(filename)
      print("Downloaded '{}' from Drive".format(filename))



    100% |████████████████████████████████| 1.4MB 16.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


Using TensorFlow backend.


    100% |████████████████████████████████| 993kB 20.6MB/s 


## Download Data 

The dataset for training can be found [here](https://download.pytorch.org/tutorial/data.zip). The trick mentioned in [fast ai forums](https://forums.fast.ai/t/request-curl-from-chrome/6816/7) has been used to download the data directly to colab instance.

In [0]:
!wget --header="Host: download.pytorch.org" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html" --header="Cookie: _ga=GA1.2.1452943855.1552129267; _gid=GA1.2.24346264.1552129267" --header="Connection: keep-alive" "https://download.pytorch.org/tutorial/data.zip" -O "data.zip" -c
!unzip data.zip

## PreProcess Downloaded Data
1.  Adding start and end of the sentence tags
2. Creating english and french tokenizers 
3. Dividing the data into training, test, validation sets 
4. Saving all the data(with divisions) as dictionary to drive for future use.  

Note: This is all one time process. The data will be downloaded for training whenever required. This helps us in maintaining the same training and validation data for training as the dataset divisions are randomized.

In [0]:
## Helper functions for preprocessing 
def process_Data(data,sequence_length):
        tokenizer = Tokenizer(oov_token='UNK',num_words = 1e4)
        tokenizer.fit_on_texts(data) 
        data_clean = tokenizer.texts_to_sequences(data)
        data_clean = pad_sequences(data_clean,sequence_length,padding='post')

        return data_clean,tokenizer
  
def read_data(filename):
  with open(filename, mode='rt', encoding='utf-8') as file:  
          dataF = file.read() 
          dataF = dataF.strip().split('\n')
          eng = []
          french = []
          for sentence in dataF: 
              split_sentence = sentence.strip().split('\t')
              sentence = "strr " + normalizeString(split_sentence[0])+ " endd"
              sentence = re.sub('\d+','num',sentence) 

              eng.append(sentence)

              sentence = "strr " + normalizeString(split_sentence[1])+ " endd"
              sentence = re.sub('\d+','num',sentence) 

              french.append(sentence) 
  return eng,french       
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    
    return s   

  
###----------------------Pre-Processing Data ----------------------------------------#####


sequence_length = 40
english_Data,french_Data = read_data("data/eng-fra.txt")
eng_data, eng_tokenizer = process_Data(english_Data,sequence_length)
french_data, french_tokenizer = process_Data(french_Data,sequence_length)


data = {}
data['eng_tokenizer']    = eng_tokenizer
data['french_tokenizer'] = french_tokenizer

x = len(eng_data)
train_idx = np.array(np.random.choice(np.arange(x),int(0.8*x),replace=False))
val_idx = np.array(np.random.choice(list(set(np.arange(x))-set(train_idx)),int(0.1*x),replace=False)) 
test_idx = list(set(np.arange(x))-set(train_idx)-set(val_idx))
print(train_idx,val_idx,test_idx)


data['eng_data_train'] = eng_data[train_idx]
data['eng_data_val'] = eng_data[val_idx]
data['eng_data_test'] = eng_data[test_idx]  

data['french_data_train'] = french_data[train_idx]
data['french_data_val'] = french_data[val_idx]
data['french_data_test'] = french_data[test_idx]  


with open("data.pkl", 'wb') as f:  
        pickle.dump(data, f)
!ls
Drive().fileaction(['data.pkl'],'up')  ## Upload the preprocess Data to drive 


[ 25034 118229  87680 ... 101026  77543 104600] [107152  39831  85160 ...  82626  87745  78961] [65537, 131074, 65541, 98313, 131083, 13, 32782, 65550, 131088, 18, 131091, 32787, 98324, 98328, 65561, 98330, 131099, 32796, 29, 98333, 98334, 98335, 32804, 98340, 98347, 98348, 32815, 65585, 32821, 65592, 65594, 65595, 61, 65597, 65598, 131136, 32834, 98374, 65607, 131145, 32846, 32848, 65622, 65624, 89, 98394, 131167, 32865, 98404, 105, 65641, 107, 32876, 98414, 32879, 131184, 32880, 131189, 118, 65653, 32889, 131194, 98425, 125, 65662, 98430, 135, 98440, 131210, 131215, 131220, 152, 131227, 32933, 98471, 169, 131241, 131246, 32942, 176, 65713, 98481, 65720, 131262, 32958, 193, 65732, 199, 98505, 131274, 131277, 98511, 98512, 32980, 32985, 32988, 32989, 222, 65757, 32995, 131302, 98535, 65768, 131312, 244, 65781, 131318, 33014, 65784, 131321, 98548, 98550, 33020, 98553, 254, 65791, 98559, 131329, 259, 65797, 262, 98567, 65801, 33035, 98572, 33037, 270, 271, 98577, 276, 131348, 33044, 9858

## Training 

### Define Encoder and Decoder Model

In [0]:
class EncoderModel(tf.keras.Model):
  def __init__(self,batch_size,embedding_dim,vocab_size,encoder_units):
    super(EncoderModel, self).__init__()
    
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.encoder_units = encoder_units
    
    self.Embedding = Embedding(self.vocab_size,self.embedding_dim,input_length=40) 
    self.GRU = CuDNNGRU(encoder_units,return_sequences = True,return_state= True) 

  def call(self, inp):
    
    embedded_vector = self.Embedding(inp)
    output,hidden_state = self.GRU(embedded_vector)
    return output,hidden_state

  def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoder_units))
    
### ----------------- Decoder Model ---------------------------------------------#####


class DecoderModel(tf.keras.Model):
  
  def __init__(self,batch_size,embedding_dim,vocab_size,decoder_units):
    super(DecoderModel, self).__init__()
    
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.decoder_units = decoder_units
    
    
    self.W1 = Dense(self.decoder_units)
    self.W2 = Dense(self.decoder_units)
    self.V  = Dense(1)
    self.A = Dense(self.decoder_units,activation='tanh')

    
    self.Embedding = Embedding(self.vocab_size,self.embedding_dim) 
    self.GRU = CuDNNGRU(self.decoder_units,return_sequences = True,return_state= True) 
    self.FC = Dense(self.vocab_size)
    
  def call(self,decoder_input,hidden,enc_hidden_output):
    
    decoder_hidden = tf.expand_dims(hidden,axis=1)                                  #output_shape = (batch_size,1,hidden_size)
    score = self.V(tf.nn.tanh(self.W1(decoder_hidden)+self.W2(enc_hidden_output)))  #output_shape = (batch_size,sequence_length,1)
    
    attention_weights = tf.nn.softmax(score,axis=1)    #output_shape = (batch_size,sequence_length,1)
    
    context = attention_weights * enc_hidden_output
    context = tf.reduce_sum(context,axis=1)            #output_shape = (batch_size,hidden_size)
    context = tf.expand_dims(context,axis=1)
    
    decoder_embedded_vector = self.Embedding(decoder_input) 
    
    attention_vector = self.A(tf.concat([context,decoder_embedded_vector],axis=-1))
    
    out,state = self.GRU(attention_vector)
    
    out = tf.squeeze(out,axis=1)
    
    output = self.FC(out)
    
    return output, state, attention_weights
    
     


### Download Processed data for training 

In [0]:
Drive().fileaction(['data.pkl'],'down')

with open("data.pkl", 'rb') as f:  
        data = pickle.load( f)
    
    
train_index = 500000
val_index = 20000
batch_size = 128

train_dataset = tf.data.Dataset.from_tensor_slices((data['eng_data_train'],data['french_data_train']))
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((data['eng_data_val'],data['french_data_val']))
val_dataset = val_dataset.batch(batch_size)



Downloaded 'data.pkl' from Drive


### Define Parameters and Instantiate Models


In [0]:
sequence_length    = 40
embedding_dim      = 128 
batch_size         = batch_size
eng_vocab_size = 15000
french_vocab_size = 15000
epochs = 5
encoder_units = 256
decoder_units = 256 
N_BATCH = data['eng_data_train'].shape[0] // batch_size
N_VAl = data['eng_data_val'].shape[0] // batch_size

learning_rate = tf.Variable(0.0008)
optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)

encoder = EncoderModel(batch_size,embedding_dim,eng_vocab_size,encoder_units)
decoder = DecoderModel(batch_size,embedding_dim,french_vocab_size,decoder_units)


def compute_loss(real, pred):
  mask = 1 - np.equal(real, 0)
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,decoder=decoder)


    


In [0]:
## Download the saved model
!rm -r training_checkpoints

Drive().fileaction(['model.zip'],'down')
!unzip -q model.zip
#!unzip -q model.zip

rm: cannot remove 'training_checkpoints': No such file or directory
Downloaded 'model.zip' from Drive


### Define Cyclic LR 

For efficient tuning of learning rate,  we are using  [Cyclical Learning Rate Policy](https://arxiv.org/abs/1506.01186)

In [0]:
def cyclic_lr(base_lr,max_lr,clr_iterations,mode,step_size,scale_fn): 
    if scale_fn == None:
            if mode == 'triangular':
                scale_fn = lambda x: 1.
                scale_mode = 'cycle'
            elif mode == 'triangular2':
                scale_fn = lambda x: 1/(2.**(x-1))
                scale_mode = 'cycle'
            elif mode == 'exp_range':
                scale_fn = lambda x: gamma**(x)
                scale_mode = 'iterations'
    else:
            scale_fn = scale_fn
    cycle = np.floor(1+clr_iterations/(2*step_size))
    x = np.abs(clr_iterations/step_size - 2*cycle + 1)
    if scale_mode == 'cycle':
       return base_lr + (max_lr-base_lr)*np.maximum(0, (1-x))*scale_fn(cycle)
    else:
       return base_lr + (max_lr-base_lr)*np.maximum(0, (1-x))*scale_fn(clr_iterations)   
    

lr = []
for i in range(100000): 
  lr.append(cyclic_lr(1e-5,4.5e-5,i,'triangular',10000,None))


#plt.plot(lr)  

### Training Loop 


In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
ite =1 
global_loss = []
for epoch in range(epochs) : 
 
    #hidden = encoder.initialize_hidden_state()
    sys.stdout.write("Epoch: {} \n".format(epoch + 1))

    start = time.time()
    val_loss = 0
    total_loss = 0
    for (batch, (X, Y)) in enumerate(train_dataset): 
          loss = 0
    
          with tf.GradientTape() as tape:

              enc_output,enc_hidden = encoder(X)
              dec_hidden = enc_hidden
              dec_input = tf.expand_dims(Y[:,0],axis=1)
              #print(Y.shape)
              for t in range(1,Y.shape[1]): 

                  dec_output,dec_hidden,_ = decoder(dec_input,dec_hidden,enc_output)

                  loss += compute_loss(Y[:,t],dec_output)

                  dec_input = tf.expand_dims(Y[:,t],axis=1)

              batch_loss = loss/batch_size
              total_loss += batch_loss

              variables = encoder.variables + decoder.variables

              gradients = tape.gradient(loss, variables)

              optimizer.apply_gradients(zip(gradients, variables))

              if(batch%3==0):
                sys.stdout.write("\r{}/{} -- Loss {:.4f} -- lr:{} ".format(
                                                              batch,N_BATCH,
                                                              batch_loss.numpy(),optimizer._lr))
                sys.stdout.flush()
                
                
              #lr = cyclic_lr(0.003,0.01,ite,'triangular',8000,None)
              global_loss.append(batch_loss)
              learning_rate.assign(lr[ite])
              ite = ite +1 
              
 
    for (batch, (X, Y)) in enumerate(val_dataset): 
              loss = 0
    
         

              enc_output,enc_hidden = encoder(X)
              dec_hidden = enc_hidden
              dec_input = tf.expand_dims(Y[:,0],axis=1)
              

              for t in range(1,Y.shape[1]): 

                  dec_output,dec_hidden,_ = decoder(dec_input,dec_hidden,enc_output)

                  loss += compute_loss(Y[:,t],dec_output)

                  dec_input = tf.expand_dims(Y[:,t],axis=1)

              batch_loss = loss/batch_size
              val_loss += batch_loss

                        
    # saving (checkpoint) the model each epoch
    if (epoch + 1) % 1== 0:
      
      !rm -r model.zip 
      !rm -r ./training_checkpoints
      checkpoint.save(file_prefix = checkpoint_prefix)
      !zip -r -qq model.zip ./training_checkpoints 
      Drive().fileaction(['model.zip'],'up')
      
    
    print("")
    sys.stdout.write("\rLoss: {:.4f} -- Val_loss: {:.4f} ".format(
                                                               total_loss / N_BATCH,val_loss/N_VAl))
    sys.stdout.flush()
   
    print('\nTime taken for 1 epoch {} sec\n'.format(time.time() - start))

    #!nvidia-smi

Epoch: 1 
849/849 -- Loss 0.0118 -- lr:<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.29715e-05> Saved 'model.zip' to Drive

Loss: 0.0853 -- Val_loss: 0.1350 
Time taken for 1 epoch 970.571409702301 sec

Epoch: 2 
849/849 -- Loss 0.0115 -- lr:<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.59465e-05> Saved 'model.zip' to Drive

Loss: 0.0850 -- Val_loss: 0.1351 
Time taken for 1 epoch 964.1758131980896 sec

Epoch: 3 
849/849 -- Loss 0.0115 -- lr:<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.89215e-05> Saved 'model.zip' to Drive

Loss: 0.0850 -- Val_loss: 0.1351 
Time taken for 1 epoch 962.2085185050964 sec

Epoch: 4 
849/849 -- Loss 0.0115 -- lr:<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.18965e-05> Saved 'model.zip' to Drive

Loss: 0.0849 -- Val_loss: 0.1351 
Time taken for 1 epoch 963.1241302490234 sec

Epoch: 5 
849/849 -- Loss 0.0114 -- lr:<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.48715e-05> Saved 'model.zip' to Drive

L

## Evaluate


In [0]:
def tokens_to_sentence(tokenizer,tokens): 
    sen ="" 
    for token in tokens: 
          word = tokenizer.index_word[token] 
       
          if(word!="strr"& word!='endd'): 
                sen += " "+ word
    return sen        
def compute_BLEU(seq,ref):
    references = [x.strip().split() for x in ref]
    #print(list(references))
    hypothesis = seq
    hypothesis = hypothesis.strip().split() 
    BLEUscore = nltk.translate.bleu_score.sentence_bleu(references, hypothesis,smoothing_function= nltk.translate.bleu_score.SmoothingFunction().method1) 
    return BLEUscore

test_dataset = tf.data.Dataset.from_tensor_slices((data['eng_data_test'],data['french_data_test']))
test_dataset = test_dataset.batch(1)

french_tokenizer = data['french_tokenizer']
eng_tokenizer = data['eng_tokenizer']

sequence_length = 40
out_seq_length = 40

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
bleu_scores = []
for (batch, (X, Y)) in enumerate(test_dataset): 
                  enc_output,enc_hidden = encoder(X)
                  dec_hidden = enc_hidden
                  seq = "strr"
                  source = ""
                  target = ""
                  tokens = french_tokenizer.texts_to_sequences([seq])
                  pad_tokens = pad_sequences(tokens,sequence_length,padding='post')[:,0]
                  dec_input = tf.expand_dims(pad_tokens,axis=1)

                  for t in range(1,out_seq_length): 

                      
                      dec_output,dec_hidden,_ = decoder(dec_input,dec_hidden,enc_output)

                      top_indices = np.argsort(-dec_output,axis=-1)
                      
                      #dec_input = 
                      #print(np.asscalar(Y[:,t].numpy()))
                      word = french_tokenizer.index_word[np.asscalar(top_indices[:,0])] 
                      if(word!="endd"): seq += " " + word 
                      if(np.asscalar(X[:,t].numpy())!=0 ): 
                        source+= " "+ eng_tokenizer.index_word[np.asscalar(X[:,t].numpy())]
                      if(np.asscalar(Y[:,t].numpy())!=0 ):   
                        target+= " "+ french_tokenizer.index_word[np.asscalar(Y[:,t].numpy())]
                      if(word!="endd"): dec_input = tf.expand_dims(top_indices[:,0],axis=1)
                  
                  
                  #print("source= ",source)    
                  #print("target= ",target)
                  #print("translated= ",seq[5:])
                  bleu  = compute_BLEU(seq[5:]+" endd",[target])

                  sys.stdout.write("\rEvaluation Progress: {}/{} -- BLEU SCORE: {:.4f} ".format(
                                                               batch,len(data['eng_data_test']),bleu))
                  sys.stdout.flush()
                  #print("BLEU SCORE =",bleu)
                  bleu_scores.append(bleu)

                  #print(return_reference(source))
print("\nAverage Bleu Score:" ,np.array(bleu_scores).mean())
                  
                  

#print(french)                      
#print(seq)                      


                 








Instructions for updating:
Colocations handled automatically by placer.
Evaluation Progress: 13584/13585 -- BLEU SCORE: 0.0309 
Average Bleu Score: 0.2663883949708836
